In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
sec = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=sec

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

f:\Programming 2\GENAI(GitHub)\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone.vectorstores import PineconeVectorStore
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get("PINECONE_API_KEY")
environment = os.environ.get("PINECONE_ENVIRONMENT")
pc = Pinecone(api_key=api_key)

# Ensure the index exists
index_name = "chatinit"  # Replace with your actual index name
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

embedding_model = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-en"
)
index = pc.Index(index_name)
vectore_Store = PineconeVectorStore(index=index, embedding=embedding_model)

f:\Programming 2\GENAI(GitHub)\venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ashiq and Adnan\.cache\huggingface\hub\models--jinaai--jina-embeddings-v2-base-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [4]:
retriever = vectore_Store.as_retriever(search_type="similarity", search_kwargs={
        "namespace": "solvrz", "k": 3
    })

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [6]:
custom_prompt = PromptTemplate(
    template="""  
Your task is to analyze the user's query and determine whether it falls under the category of "sales," "support," or is a "general" query. 
Based on the category, respond accordingly, considering the context of the ongoing conversation.

**Context:** 
{context}
**User Query:**
{input}

**Instructions:**
1. **Categorization:**
   - Analyze the query and classify it as "sales," "support," or "general."
2. **Response Based on Category:**
   - If the query relates to sales, provide a sales-specific answer.
   - If it relates to support, offer guidance or troubleshooting steps.
   - For general queries, give helpful insights.
3. **Tone and Style:**
   - Use a professional and friendly tone.
   - Keep responses concise and contextually relevant.
   -Avoid unnecessary or filler phrases that do not add value to the response. Focus on delivering concise and contextually relevant answer
Just response the answer without mentioning the type of query.
""",
    input_variables=["input", "context"],
)


In [7]:
combine_docs_chain = create_stuff_documents_chain(llm, custom_prompt)

# Step 3: Create the retrieval chain using your custom chain
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [11]:
enabled_services = "sales, support, feedback, appointment booking, learn more"
query = "What are the product of Solvrz?"
chat_history = []
context = "Relevant documents retrieved based on the user's query."

In [12]:
response = rag_chain.invoke({
    "input": query,
    "context": context
})


In [13]:
answer = response.get("answer", "No answer found")
print(answer)

At Solvrz, we empower businesses through digital transformation.  Our expertise lies in research and innovation, pioneering advancements with frontier technology research and development services. We leverage emerging technologies like Machine Learning, AI automation, Data Analysis, Blockchain, IoT, and Cloud Computing to craft innovative solutions.  We're also adept at building intellectual property tracking systems using smart contracts and blockchain technology.

